In [20]:
from tensorflow import keras
from keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.utils import image_dataset_from_directory

In [9]:
train_dataset = image_dataset_from_directory(
    directory = './train',
    labels = 'inferred',
    label_mode = 'binary',
    batch_size = 64,
    image_size = (224,224)
)

test_dataset = image_dataset_from_directory(
    directory = './test',
    labels = 'inferred',
    label_mode = 'binary',
    batch_size = 64,
    image_size = (224,224)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [10]:
#normalize images to [0,1]
def norm_img(image, label):
    return image/255, label

train_dataset = train_dataset.map(norm_img)
test_dataset = test_dataset.map(norm_img)

In [17]:
base_model = VGG16(weights='imagenet', include_top= False, input_shape=(224, 224, 3))


In [18]:
for layer in base_model.layers:
    layer.trainable = False

In [22]:
x = base_model.output
x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dense(1,activation='sigmoid')(x)

new_model = Model(inputs = base_model.input, outputs = x)

In [23]:
new_model.compile(optimizer=Adam(learning_rate=0.001), metrics= ['accuracy'], loss= 'binary_crossentropy')

In [24]:
history = new_model.fit(train_dataset, validation_data= test_dataset, epochs= 5)

Epoch 1/5


I0000 00:00:1742449735.763238   18794 service.cc:148] XLA service 0x7f3efc00caf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742449735.765447   18794 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2025-03-20 11:18:55.854473: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742449736.439521   18794 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-20 11:18:59.246412: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-03-20 11:18:59.246564: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU

313/313 ━━━━━━━━━━━━━━━━━━━━ 132s 348ms/step - accuracy: 0.8292 - loss: 0.5125 - val_accuracy: 0.9068 - val_loss: 0.2096
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 94s 300ms/step - accuracy: 0.9414 - loss: 0.1424 - val_accuracy: 0.9368 - val_loss: 0.1556
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 143s 303ms/step - accuracy: 0.9582 - loss: 0.1059 - val_accuracy: 0.9268 - val_loss: 0.1877
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 93s 297ms/step - accuracy: 0.9684 - loss: 0.0833 - val_accuracy: 0.9352 - val_loss: 0.1773
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 94s 301ms/step - accuracy: 0.9786 - loss: 0.0572 - val_accuracy: 0.9134 - val_loss: 0.2566


In [26]:
_ , accuracy = new_model.evaluate(test_dataset)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 228ms/step - accuracy: 0.9119 - loss: 0.2609
Validation Accuracy: 91.34%


In [28]:
base_model_resnet = ResNet50(include_top=False, weights= 'imagenet', input_shape=(224,224,3))

for layer in base_model_resnet.layers:
    layer.trainable = False

In [29]:
x = base_model_resnet.output

x = Flatten()(x)
x = Dense(128, activation = 'relu')(x)
x = Dense(1, activation= 'sigmoid')(x)

new_model2 = Model(inputs = base_model_resnet.input, outputs = x)

In [31]:
new_model2.compile(optimizer=Adam(learning_rate = 0.001), metrics = ['accuracy'], loss = 'binary_crossentropy')

In [32]:
history = new_model2.fit(train_dataset, epochs = 5, validation_data= test_dataset )

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 86s 234ms/step - accuracy: 0.5685 - loss: 1.5796 - val_accuracy: 0.6522 - val_loss: 0.6166
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 63s 201ms/step - accuracy: 0.6690 - loss: 0.6070 - val_accuracy: 0.6768 - val_loss: 0.5994
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 62s 199ms/step - accuracy: 0.6953 - loss: 0.5784 - val_accuracy: 0.7202 - val_loss: 0.5533
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 60s 193ms/step - accuracy: 0.6954 - loss: 0.5776 - val_accuracy: 0.6772 - val_loss: 0.5940
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 59s 188ms/step - accuracy: 0.7145 - loss: 0.5533 - val_accuracy: 0.7366 - val_loss: 0.5349


In [33]:
_ , accuracy = new_model2.evaluate(test_dataset)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

79/79 ━━━━━━━━━━━━━━━━━━━━ 12s 152ms/step - accuracy: 0.7382 - loss: 0.5309
Validation Accuracy: 73.66%
